# [Modelling target]

:::{eval-rst}
:opticon:`tag`
:badge:`[Environment],badge-primary`
:badge:`Modelling,badge-secondary`
:::

## Context
### Purpose
*Describe the purpose of the use case.*

### Modelling approach
*Describe the most relevant features of the modelling approach.*

### Highlights
*Provide 3-5 bullet points that convey the use case’s core procedures. Each bullet point must have a maximum of 85 characters, including spaces.*
* Highlight 1
* Highlight 2

### Contributions
#### Notebook
Author (role), Affiliation, GitHub alias

#### Modelling codebase
Author (role), Affiliation, GitHub alias

#### Modelling publications
```{bibliography}
  :style: plain
  :list: bullet
  :filter: topic % "replace by the `topic` entry linked to the publication(s) in the `_bibliography/references.bib` file"
```

#### Modelling funding
Indicate details of funding.

:::{note}
*Optional: add credits or acknowledgements to data providers or authors of code snippets*
:::

## Install and load libraries

## Load data

```python
import logging
logging.basicConfig(level=logging.INFO)
from icenet2.data.interfaces.cds import ERA5Downloader
era5 = ERA5Downloader(
    var_names=["tos"],
    pressure_levels=[None],
    dates=[pd.to_datetime(date).date() for date in
           pd.date_range("1990-1-1", "1990-1-31",
                         freq="D")],
    delete_tempfiles=False,
    max_threads=1,
    north=False,
    south=True,
    use_toolbox=True
)
era5.download()
era5.regrid()
era5.rotate_wind_data()
```

## Outputs

## Summary

## Version
- Notebook:
- Codebase: